# Jointure MapReduce

In [2]:
%AddJar file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar

Using cached version of Emiasd-Flight-Data-Analysis.jar
Using cached version of Emiasd-Flight-Data-Analysis.jar


In [3]:
import org.apache.spark.sql.SparkSession
import com.flightdelay.config.{AppConfiguration, ConfigurationLoader}
import com.flightdelay.data.loaders.FlightDataLoader

//Env Configuration
val args: Array[String] = Array("juniper")

val spark = SparkSession.builder()
  .config(sc.getConf)
  .getOrCreate()

// Rendre la session Spark implicite
implicit val session = spark
implicit val configuration: AppConfiguration = ConfigurationLoader.loadConfiguration(args)

args = Array(juniper)
spark = org.apache.spark.sql.SparkSession@31131074
session = org.apache.spark.sql.SparkSession@31131074
configuration = AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/201201hourly.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileCo...


AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/201201hourly.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileCo...

# Chargement Flight et Weather

In [4]:
// Chargement de Flight Data
val processedFlightPath = s"${configuration.common.output.basePath}/common/data/processed_flights.parquet"
val flightDF = spark.read.parquet(processedFlightPath)

val processedWeatherPath = s"${configuration.common.output.basePath}/common/data/processed_weather.parquet"
val weatherDF = spark.read.parquet(processedWeatherPath)



processedFlightPath = /home/jovyan/work/output/common/data/processed_flights.parquet
flightDF = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 97 more fields]
processedWeatherPath = /home/jovyan/work/output/common/data/processed_weather.parquet
weatherDF = [WBAN: string, Date: date ... 65 more fields]


[WBAN: string, Date: date ... 65 more fields]

In [5]:
weatherDF.count()

489665

In [6]:
weatherDF.printSchema

root
 |-- WBAN: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: string (nullable = true)
 |-- StationType: integer (nullable = true)
 |-- SkyCondition: string (nullable = true)
 |-- SkyConditionFlag: string (nullable = true)
 |-- Visibility: double (nullable = true)
 |-- VisibilityFlag: string (nullable = true)
 |-- WeatherType: string (nullable = true)
 |-- WeatherTypeFlag: string (nullable = true)
 |-- DryBulbFarenheit: double (nullable = true)
 |-- DryBulbFarenheitFlag: string (nullable = true)
 |-- DryBulbCelsius: double (nullable = true)
 |-- DryBulbCelsiusFlag: string (nullable = true)
 |-- WetBulbFarenheit: double (nullable = true)
 |-- WetBulbFarenheitFlag: string (nullable = true)
 |-- WetBulbCelsius: double (nullable = true)
 |-- WetBulbCelsiusFlag: string (nullable = true)
 |-- DewPointFarenheit: double (nullable = true)
 |-- DewPointFarenheitFlag: string (nullable = true)
 |-- DewPointCelsius: double (nullable = true)
 |-- DewPointCelsiusFlag: string (

In [7]:
val weatherColumns = Seq(
  "WBAN",
  "Date",
  "Time",
  "StationType",
  "SkyCondition",
  "Visibility",

  "feature_ceiling",  
  /**"feature_most_critical_sky",
  "feature_lowest_cloud_height",
  
  "feature_num_cloud_layers",
  "feature_cloud_risk_score",
  "feature_has_overcast",
  "feature_has_broken",
  "feature_has_obscured",
  "feature_is_clear",
  "feature_has_low_ceiling",
  "feature_visibility_miles",
  "feature_visibility_km",
  "feature_visibility_category",
  "feature_visibility_risk_score",
  "feature_is_low_visibility",
  "feature_is_very_low_visibility",
  "feature_visibility_normalized",
  "feature_visibility_inverse",
  "feature_weather_severity_index",
  "feature_is_vfr_conditions",
  "feature_is_ifr_conditions",
  "feature_requires_cat_ii",
  "feature_operations_risk_level"**/
)

val weatherData = weatherDF
  .select(weatherColumns.head, weatherColumns.tail: _*)
  .orderBy("WBAN", "Date", "Time")
weatherData.show(10)


+-----+----------+----+-----------+------------+----------+---------------+
| WBAN|      Date|Time|StationType|SkyCondition|Visibility|feature_ceiling|
+-----+----------+----+-----------+------------+----------+---------------+
|03011|2012-01-01|0000|          0|         CLR|      10.0|          99999|
|03011|2012-01-01|0100|          0|         CLR|      10.0|          99999|
|03011|2012-01-01|0200|          0|         CLR|      10.0|          99999|
|03011|2012-01-01|0300|          0|         CLR|      10.0|          99999|
|03011|2012-01-01|0400|          0|         CLR|      10.0|          99999|
|03011|2012-01-01|0500|          0|         CLR|      10.0|          99999|
|03011|2012-01-01|0600|          0|         CLR|      10.0|          99999|
|03011|2012-01-01|0700|          0|         CLR|      10.0|          99999|
|03011|2012-01-01|0800|          0|         CLR|      10.0|          99999|
|03011|2012-01-01|0900|          0|         CLR|      10.0|          99999|
+-----+-----

weatherColumns = List(WBAN, Date, Time, StationType, SkyCondition, Visibility, feature_ceiling)
weatherData = [WBAN: string, Date: date ... 5 more fields]


[WBAN: string, Date: date ... 5 more fields]

In [8]:
flightDF.count()

134665

In [9]:
flightDF.printSchema

root
 |-- FL_DATE: date (nullable = true)
 |-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN_AIRPORT_ID: integer (nullable = true)
 |-- DEST_AIRPORT_ID: integer (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- ARR_DELAY_NEW: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- WEATHER_DELAY: double (nullable = true)
 |-- NAS_DELAY: double (nullable = true)
 |-- ORIGIN_WBAN: string (nullable = true)
 |-- ORIGIN_TIMEZONE: integer (nullable = true)
 |-- DEST_WBAN: string (nullable = true)
 |-- DEST_TIMEZONE: integer (nullable = true)
 |-- UTC_CRS_DEP_TIME: string (nullable = true)
 |-- UTC_FL_DATE: date (nullable = true)
 |-- feature_departure_minute: integer (nullable = true)
 |-- feature_flight_days_span: integer (nullable = true)
 |-- feature_arrival_minutes_total: double (nullable = true)
 |-- feature_crosses_midnight: integer (nullable = true)
 |-- feature_flight_day_of_week: 

In [10]:
val flightColumns = Seq(
  "UTC_FL_DATE",
  "UTC_CRS_DEP_TIME",
  "ORIGIN_WBAN",
  "DEST_WBAN",
  "CRS_ELAPSED_TIME",
  "feature_utc_departure_hour",
  "OP_CARRIER_AIRLINE_ID",
  "OP_CARRIER_FL_NUM",  
  //"feature_flight_month",
  //"feature_flight_year",
  //"feature_flight_quarter",
  "feature_flight_day_of_week",
  "feature_is_weekend",  
  "feature_utc_departure_hour_rounded",  
  "label_is_delayed_15min"
)

val flightData = flightDF
  .select(flightColumns.head, flightColumns.tail: _*)
flightData.show(10)


+-----------+----------------+-----------+---------+----------------+--------------------------+---------------------+-----------------+--------------------------+------------------+----------------------------------+----------------------+
|UTC_FL_DATE|UTC_CRS_DEP_TIME|ORIGIN_WBAN|DEST_WBAN|CRS_ELAPSED_TIME|feature_utc_departure_hour|OP_CARRIER_AIRLINE_ID|OP_CARRIER_FL_NUM|feature_flight_day_of_week|feature_is_weekend|feature_utc_departure_hour_rounded|label_is_delayed_15min|
+-----------+----------------+-----------+---------+----------------+--------------------------+---------------------+-----------------+--------------------------+------------------+----------------------------------+----------------------+
| 2012-01-09|            1655|      13869|    13874|            57.0|                        16|                20366|             5334|                         2|                 0|                              1700|                     1|
| 2012-01-08|            2230|      

flightColumns = List(UTC_FL_DATE, UTC_CRS_DEP_TIME, ORIGIN_WBAN, DEST_WBAN, CRS_ELAPSED_TIME, feature_utc_departure_hour, OP_CARRIER_AIRLINE_ID, OP_CARRIER_FL_NUM, feature_flight_day_of_week, feature_is_weekend, feature_utc_departure_hour_rounded, label_is_delayed_15min)
flightData = [UTC_FL_DATE: date, UTC_CRS_DEP_TIME: string ... 10 more fields]


[UTC_FL_DATE: date, UTC_CRS_DEP_TIME: string ... 10 more fields]

# Jointure

In [11]:
import com.flightdelay.data.joiners.FlightWeatherDataJoiner

val joinOriginDataSet = FlightWeatherDataJoiner.joinOriginWeather(flightData, weatherData)


  [Origin Weather Join] Weather columns to join: 4
    - Weather features: StationType, SkyCondition, Visibility, feature_ceiling


joinOriginDataSet = [UTC_FL_DATE: date, UTC_CRS_DEP_TIME: string ... 16 more fields]


  [Origin Weather Join] Completed in 0.095s - Created 4 weather arrays


[UTC_FL_DATE: date, UTC_CRS_DEP_TIME: string ... 16 more fields]

In [12]:
joinOriginDataSet.show(10)

+-----------+----------------+-----------+---------+----------------+--------------------------+--------------------+-------------------+----------------------+--------------------------+------------------+----------------------------------+----------------------+--------------------------------+---------------------------------+-------------------------------+------------------------------------+------------------------+
|UTC_FL_DATE|UTC_CRS_DEP_TIME|ORIGIN_WBAN|DEST_WBAN|CRS_ELAPSED_TIME|feature_utc_departure_hour|feature_flight_month|feature_flight_year|feature_flight_quarter|feature_flight_day_of_week|feature_is_weekend|feature_utc_departure_hour_rounded|label_is_delayed_15min|origin_weather_StationType_array|origin_weather_SkyCondition_array|origin_weather_Visibility_array|origin_weather_feature_ceiling_array|origin_weather_obs_count|
+-----------+----------------+-----------+---------+----------------+--------------------------+--------------------+-------------------+-----------

# Jointure Map Reduce

In [11]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.Row
import scala.collection.JavaConverters._

// Recommandé: travailler en UTC pour éviter les surprises
spark.conf.set("spark.sql.session.timeZone", "UTC")
spark.conf.set("spark.sql.crossJoin.enabled", "true")
spark.conf.set("spark.sql.debug.maxToStringFields", "100000")


In [12]:
// -----------------------------------------
// 0) Normalisation du dataset Weather
//    - DATE  -> date
//    - TIME  -> heure 2 chiffres "HH"
// -----------------------------------------
val weatherNorm = weatherData
  // DATE peut être string => on force en date (UTC)
  .withColumn("W_DATE", to_date(col("DATE")))
  // TIME peut être "0", "00", "0030", 0, etc.
  .withColumn("TIME_STR", col("TIME").cast("string"))
  .withColumn(
    "W_HOUR",
    when(length(col("TIME_STR")) >= 2, substring(col("TIME_STR"), 1, 2))
      .otherwise(lpad(col("TIME_STR"), 2, "0"))
  )
  .select(
    col("WBAN"),
    col("W_DATE"),
    col("W_HOUR"),
    // garde ce qui t'intéresse côté météo :
    col("*") // tu peux remplacer par les colonnes utiles: vent, visib, etc.
  )
weatherNorm.show(10)

+-----+----------+------+-----+----------+----+-----------+------------+----------+---------------+----------+--------+------+
| WBAN|    W_DATE|W_HOUR| WBAN|      Date|Time|StationType|SkyCondition|Visibility|feature_ceiling|    W_DATE|TIME_STR|W_HOUR|
+-----+----------+------+-----+----------+----+-----------+------------+----------+---------------+----------+--------+------+
|03011|2012-01-01|    00|03011|2012-01-01|0000|          0|         CLR|      10.0|          99999|2012-01-01|    0000|    00|
|03011|2012-01-01|    01|03011|2012-01-01|0100|          0|         CLR|      10.0|          99999|2012-01-01|    0100|    01|
|03011|2012-01-01|    02|03011|2012-01-01|0200|          0|         CLR|      10.0|          99999|2012-01-01|    0200|    02|
|03011|2012-01-01|    03|03011|2012-01-01|0300|          0|         CLR|      10.0|          99999|2012-01-01|    0300|    03|
|03011|2012-01-01|    04|03011|2012-01-01|0400|          0|         CLR|      10.0|          99999|2012-01-01| 

weatherNorm = [WBAN: string, W_DATE: date ... 11 more fields]


[WBAN: string, W_DATE: date ... 11 more fields]

In [13]:
// Option: pour le shuffle, tu peux aider Spark à partitionner:
val weatherPart = weatherNorm.repartition(col("WBAN"), col("W_DATE"))

weatherPart = [WBAN: string, W_DATE: date ... 11 more fields]


[WBAN: string, W_DATE: date ... 11 more fields]

In [14]:
// ───────────────────────────────────────────────────────────────────────────────
// 0) Crée un identifiant de vol déterministe "flight_id"
//    (tu peux remplacer par monotonically_increasing_id() si tu préfères)
// ───────────────────────────────────────────────────────────────────────────────
val flights0 = flightData
  .withColumn(
    "flight_id",
    concat_ws("_",
      date_format(col("UTC_FL_DATE").cast("date"), "yyyyMMdd"),
      col("OP_CARRIER_AIRLINE_ID").cast("string"),
      col("OP_CARRIER_FL_NUM").cast("string"),
      col("ORIGIN_WBAN").cast("string"),
      col("DEST_WBAN").cast("string"),
      date_format(col("UTC_CRS_DEP_TIME"), "HHmm")
    )
  )

flights0.show(10)

+-----------+----------------+-----------+---------+----------------+--------------------------+---------------------+-----------------+--------------------------+------------------+----------------------------------+----------------------+--------------------+
|UTC_FL_DATE|UTC_CRS_DEP_TIME|ORIGIN_WBAN|DEST_WBAN|CRS_ELAPSED_TIME|feature_utc_departure_hour|OP_CARRIER_AIRLINE_ID|OP_CARRIER_FL_NUM|feature_flight_day_of_week|feature_is_weekend|feature_utc_departure_hour_rounded|label_is_delayed_15min|           flight_id|
+-----------+----------------+-----------+---------+----------------+--------------------------+---------------------+-----------------+--------------------------+------------------+----------------------------------+----------------------+--------------------+
| 2012-01-09|            1655|      13869|    13874|            57.0|                        16|                20366|             5334|                         2|                 0|                              17

flights0 = [UTC_FL_DATE: date, UTC_CRS_DEP_TIME: string ... 11 more fields]


[UTC_FL_DATE: date, UTC_CRS_DEP_TIME: string ... 11 more fields]

In [15]:

// ───────────────────────────────────────────────────────────────────────────────
// 1) Base temporelle départ (UTC) : "yyyy-MM-dd HH" → epoch (secondes)
// ───────────────────────────────────────────────────────────────────────────────
val flightsBase = flights0
  .withColumn(
    "FL_DATE_STR",
    when(col("UTC_FL_DATE").cast("date").isNotNull,
         date_format(col("UTC_FL_DATE").cast("date"), "yyyy-MM-dd"))
    .otherwise(col("UTC_FL_DATE").cast("string"))
  )
  .withColumn("DEP_HOUR_2D", lpad(col("feature_utc_departure_hour_rounded").cast("string"), 2, "0"))
  .withColumn("base_epoch",
    unix_timestamp(concat_ws(" ", col("FL_DATE_STR"), col("DEP_HOUR_2D")), "yyyy-MM-dd HH")
  )
  // IMPORTANT : on ne garde QUE ce qu'on va utiliser ensuite (dont flight_id !)
  .select(
    col("flight_id"),
    col("ORIGIN_WBAN"),
    col("DEST_WBAN"),
    col("FL_DATE_STR"),
    col("DEP_HOUR_2D"),
    col("base_epoch"),
    col("label_is_delayed_15min")  
  )

flightsBase.show(10)

+--------------------+-----------+---------+-----------+-----------+----------+----------------------+
|           flight_id|ORIGIN_WBAN|DEST_WBAN|FL_DATE_STR|DEP_HOUR_2D|base_epoch|label_is_delayed_15min|
+--------------------+-----------+---------+-----------+-----------+----------+----------------------+
|20120109_20366_53...|      13869|    13874| 2012-01-09|         17|1326128400|                     1|
|20120108_20366_53...|      13869|    13874| 2012-01-08|         23|1326063600|                     1|
|20120108_20366_53...|      13869|    13874| 2012-01-08|         17|1326042000|                     1|
|20120104_20366_53...|      13869|    13874| 2012-01-04|         23|1325718000|                     1|
|20120122_20366_53...|      13869|    13874| 2012-01-22|         17|1327251600|                     1|
|20120110_20366_53...|      13869|    13874| 2012-01-10|         23|1326236400|                     1|
|20120121_20366_53...|      13869|    13874| 2012-01-21|         17|13271

flightsBase = [flight_id: string, ORIGIN_WBAN: string ... 5 more fields]


[flight_id: string, ORIGIN_WBAN: string ... 5 more fields]

In [16]:
// ───────────────────────────────────────────────────────────────────────────────
// 2) Générer les 12 heures cibles t-1h..t-12h (crossJoin offsets 1..12)
// ───────────────────────────────────────────────────────────────────────────────
val offsets = flightsBase.sparkSession.range(1, 13).toDF("h")  // 1..12

val originTargets = flightsBase
  .crossJoin(offsets)
  .withColumn("target_epoch", col("base_epoch") - col("h") * lit(3600))
  .withColumn("TARGET_DATE", to_date(from_unixtime(col("target_epoch"))))
  .withColumn("TARGET_HOUR_2D", date_format(from_unixtime(col("target_epoch")), "HH"))
  .select(
    col("flight_id"),                                    // ← remplace flightId par flight_id
    col("ORIGIN_WBAN").as("WBAN_KEY"),
    col("h").cast("int").as("o_h"),
    col("TARGET_DATE"),
    col("TARGET_HOUR_2D"),
    col("label_is_delayed_15min")  
  )

// (debug)
originTargets.show(10, truncate = false)

+------------------------------------+--------+---+-----------+--------------+----------------------+
|flight_id                           |WBAN_KEY|o_h|TARGET_DATE|TARGET_HOUR_2D|label_is_delayed_15min|
+------------------------------------+--------+---+-----------+--------------+----------------------+
|20120109_20366_5334_13869_13874_0000|13869   |1  |2012-01-09 |16            |1                     |
|20120109_20366_5334_13869_13874_0000|13869   |2  |2012-01-09 |15            |1                     |
|20120109_20366_5334_13869_13874_0000|13869   |3  |2012-01-09 |14            |1                     |
|20120109_20366_5334_13869_13874_0000|13869   |4  |2012-01-09 |13            |1                     |
|20120109_20366_5334_13869_13874_0000|13869   |5  |2012-01-09 |12            |1                     |
|20120109_20366_5334_13869_13874_0000|13869   |6  |2012-01-09 |11            |1                     |
|20120109_20366_5334_13869_13874_0000|13869   |7  |2012-01-09 |10            |1   

offsets = [h: bigint]
originTargets = [flight_id: string, WBAN_KEY: string ... 4 more fields]


[flight_id: string, WBAN_KEY: string ... 4 more fields]

In [31]:
// ───────────────────────────────────────────────────────────────────────────────
// 3) Normalisation météo (si pas déjà fait) et jointure exacte (WBAN, DATE, HOUR)
// ───────────────────────────────────────────────────────────────────────────────

val originJoined = originTargets
  .join(
    weatherNorm,
    originTargets.col("WBAN_KEY") === weatherNorm.col("WBAN") &&
    originTargets.col("TARGET_DATE") === weatherNorm.col("W_DATE") &&
    originTargets.col("TARGET_HOUR_2D") === weatherNorm.col("W_HOUR"),
    "inner"
  )
  .select(
    col("flight_id"),
    col("o_h"),
    col("WBAN_KEY").as("WBAN"),
    col("TARGET_DATE").as("o_target_date"),
    col("TARGET_HOUR_2D").as("o_target_hour"),
    col("W_DATE"), col("W_HOUR"),
    col("DATE"), col("TIME"),
    col("StationType"),
    col("SkyCondition"),
    col("Visibility"),
    col("feature_ceiling"),  
  ).orderBy(col("flight_id"), col("o_target_hour"))

originJoined.show(20, false)

+------------------------------------+---+-----+-------------+-------------+----------+------+----------+----+-----------+--------------------+----------+---------------+
|flight_id                           |o_h|WBAN |o_target_date|o_target_hour|W_DATE    |W_HOUR|DATE      |TIME|StationType|SkyCondition        |Visibility|feature_ceiling|
+------------------------------------+---+-----+-------------+-------------+----------+------+----------+----+-----------+--------------------+----------+---------------+
|20120101_19393_1035_14839_23183_0000|8  |14839|2012-01-01   |06           |2012-01-01|06    |2012-01-01|0600|11         |BKN016 BKN026 OVC032|5.0       |1600           |
|20120101_19393_122_23188_93721_0000 |5  |23188|2012-01-01   |10           |2012-01-01|10    |2012-01-01|1000|11         |FEW001              |3.0       |99999          |
|20120101_19393_1306_23188_23169_0000|10 |23188|2012-01-01   |10           |2012-01-01|10    |2012-01-01|1000|11         |FEW001              |3.

originJoined = [flight_id: string, o_h: int ... 11 more fields]


[flight_id: string, o_h: int ... 11 more fields]

In [19]:
// ───────────────────────────────────────────────────────────────────────────────
// 4) (Option) Regrouper en liste par vol (gardez o_h pour trier côté lecteur)
// ───────────────────────────────────────────────────────────────────────────────
val originAgg = originJoined
  .withColumn("o_pack", struct(
    col("o_h"),
    col("o_target_date"),
    col("o_target_hour"),
    col("W_DATE"),
    col("W_HOUR")
    // + tes variables météo utiles
  ))
  .groupBy("flight_id")                           // ← remplace "flightId"
  .agg(collect_list(col("o_pack")).as("W_o_list"))

originAgg.show(10)

+--------------------+--------------------+
|           flight_id|            W_o_list|
+--------------------+--------------------+
|20120101_19393_10...|[{8, 2012-01-01, ...|
|20120101_19393_12...|[{5, 2012-01-01, ...|
|20120101_19393_13...|[{10, 2012-01-01,...|
|20120101_19393_13...|[{6, 2012-01-01, ...|
|20120101_19393_14...|[{8, 2012-01-01, ...|
|20120101_19393_15...|[{8, 2012-01-01, ...|
|20120101_19393_16...|[{9, 2012-01-01, ...|
|20120101_19393_18...|[{7, 2012-01-01, ...|
|20120101_19393_19...|[{5, 2012-01-01, ...|
|20120101_19393_23...|[{5, 2012-01-01, ...|
+--------------------+--------------------+
only showing top 10 rows



originAgg = [flight_id: string, W_o_list: array<struct<o_h:int,o_target_date:date,o_target_hour:string,W_DATE:date,W_HOUR:string>>]


[flight_id: string, W_o_list: array<struct<o_h:int,o_target_date:date,o_target_hour:string,W_DATE:date,W_HOUR:string>>]

In [20]:
import org.apache.spark.sql.functions._

originAgg.limit(1)
  .select(to_json(struct(originAgg.columns.map(col): _*)).as("json"))
  .collect()
  .foreach(r => println(r.getAs[String]("json")))

{"flight_id":"20120101_19393_1035_14839_23183_0000","W_o_list":[{"o_h":8,"o_target_date":"2012-01-01","o_target_hour":"06","W_DATE":"2012-01-01","W_HOUR":"06"}]}


# Quelques Requettes

In [28]:
import org.apache.spark.sql.functions._

// 1) Sélectionner les colonnes WBAN uniques dans chaque dataset
val flightWban = flightData.select("ORIGIN_WBAN").distinct()
val weatherWban = weatherData.select("WBAN").distinct()

// 2) Faire une "left anti join" → garde ceux qui ne matchent PAS
val missingWban = flightWban.join(weatherWban,
  flightWban("ORIGIN_WBAN") === weatherWban("WBAN"),
  "left_anti"
)

// 3) Compter combien il y en a
println("Nombre de WBAN présents dans flights.ORIGIN_WBAN mais absents de WeatherData.WBAN :")
missingWban.count()

// 4) (Option) afficher la liste
missingWban.show(20, false)

Nombre de WBAN présents dans flights.ORIGIN_WBAN mais absents de WeatherData.WBAN :
+-----------+
|ORIGIN_WBAN|
+-----------+
|3927       |
|3181       |
|23259      |
|12849      |
|93184      |
|3856       |
|3822       |
|23195      |
|22521      |
|93138      |
|23183      |
|24132      |
|12894      |
|94908      |
|24090      |
|23199      |
|12816      |
|4825       |
|3947       |
|3928       |
+-----------+
only showing top 20 rows



flightWban = [ORIGIN_WBAN: string]
weatherWban = [WBAN: string]
missingWban = [ORIGIN_WBAN: string]


[ORIGIN_WBAN: string]